# Import Library

In [61]:
import pandas as pd

# Import Data

In [62]:
#OurData
dfOur = pd.read_csv("..")

#UserData
dfUser = pd.read_excel("..")

#Billing Number Initial FENNY data
dfFenny = pd.read_excel("Tissue Direct 2008-2019.xlsx")

In [141]:
dfUser.columns

Index(['Periods', 'S+R', 'Material', 'CHA', 'CHANNEL', 'RETAIL/AFH', 'STORE',
       'Customer', 'Cust Name', 'NW (Ton)', 'Value (IDR)', 'Value (USD)',
       'Ship To', 'Ship To Name', 'Qty (Karton)', 'Sales Person', 'SO No',
       'GI Date', 'RDD', 'B_ArtCode', 'B_Carton Code', 'B_Material Text',
       'B_Brand', 'B_Brand-2', 'B_Category', 'B_Category-2', 'B_Category-3',
       'B_StatusSKU', 'B_Mill', 'B_Zone', 'B_Zone-2', 'B_Zone-3', 'B_Town',
       'D_NOND', 'B_Week', 'B_Week Rdd'],
      dtype='object')

In [142]:
#Inspection Our Data
print("Our Data Dimension  ", dfOur.shape)
print("Our Null Data  ",dfOur.isnull().sum())
print("Our Data Types ", dfOur.dtypes)
print("Our Data Columns ",dfOur.columns)

Our Data Dimension   (1975994, 45)
Our Null Data   KEY_SR                                    0
Act.goods issue date                      0
Billing item                              0
Sales document                            0
Sales document item                       0
Billing number                            0
Cancelled                           1954989
Sales organization                        0
Brand ID                                  0
Brand (Group)                        238041
Order Type                                0
GSM                                       0
Material ID                               0
Manufacturing Mill                       77
Mill                                      0
Material Classification                   0
Matl Group                                0
Material Division                         0
Req. Delivery Dt/ESD                      0
Ship to Party                             0
Country (Ship To)                      1312
Sold to Party            

In [143]:
#Inspection Our Data
print("User Data Dimension  ", dfUser.shape)
print("User Null Data  ",dfUser.isnull().sum())
print("User Data Types ", dfUser.dtypes)
print("User Data Columns ",dfUser.columns)

User Data Dimension   (366887, 36)
User Null Data   Periods              0
S+R                  0
Material             0
CHA                  0
CHANNEL              0
RETAIL/AFH           0
STORE                0
Customer             0
Cust Name            0
NW (Ton)             0
Value (IDR)          0
Value (USD)          0
Ship To              0
Ship To Name         0
Qty (Karton)         0
Sales Person        24
SO No                0
GI Date              0
RDD                  0
B_ArtCode          474
B_Carton Code      198
B_Material Text    474
B_Brand             37
B_Brand-2           37
B_Category           0
B_Category-2         0
B_Category-3         0
B_StatusSKU         59
B_Mill               0
B_Zone               0
B_Zone-2             0
B_Zone-3             0
B_Town               0
D_NOND               0
B_Week               0
B_Week Rdd          23
dtype: int64
User Data Types  Periods             object
S+R                 object
Material             int64
CHA      

# Data Engineering 

## Our Data Engineering 

In [144]:
#Before we do filtering, we have take out all of Billing number of dfFenny

#Change data type of Billing number of dfOur to object
dfOur['Billing number'] = dfOur['Billing number'].astype(str)

#First beforeTake out billing number which there are in dfFenny 
dfOur2 = dfOur.loc[pd.merge(dfOur, dfFenny, on=['Billing number'], how='left', indicator=True)['_merge'] == 'left_only']

In [145]:
#One of the steps to compare our data and user data is Value (IDR), so we need add new Value (IDR) column, beacuse we haven't
#before
dfOur2['Value (IDR)']=dfOur2['Inv Sell Val Doc Cur']-dfOur2['Disc. Amount(Doc. Curr.)']

C:\Users\01135710\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [146]:
#Take features that we need
dfOur2 = dfOur2[['Act.goods issue date','Material ID','Req. Delivery Dt/ESD','Ship to Party','Sold to Party','Inv. Weight (MT)',
       'Billing number','Value (IDR)']]

In [147]:
#Change of each features name 
dfOur2.columns = ['GI Date', 'Material','Req. Delivery Dt/ESD','Ship To', 'Customer','NW (Ton)','Billing number','Value (IDR)']

In [148]:
#Check GI feature
dfOur2['GI Date'].value_counts()

#Take out 00000000 value
dfOur2 = dfOur2[dfOur2['GI Date'] != '00000000'] 

In [149]:
#Change data type of GI Date to Date
dfOur2['GI Date'] = pd.to_datetime(dfOur2['GI Date'],dayfirst=True) 

In [150]:
dfOur2.head()

,GI Date,Material,Req. Delivery Dt/ESD,Ship To,Customer,NW (Ton),Billing number,Value (IDR)
0,2008-01-02,69907062,02.01.2008,2029773,2027133,0.80000,1730002294,0.0
1,2008-01-02,69906182,02.01.2008,2029773,2027133,0.38640,1730002295,0.0
2,2008-01-02,69906662,02.01.2008,1000991,1000991,0.09005,2440004658,1064700.0
3,2008-01-02,69906431,02.01.2008,1000991,1000991,0.25135,2440004658,3648250.0
4,2008-01-02,69906182,02.01.2008,2029773,2027133,0.10080,1730002294,0.0


## User Data Engineering

In [151]:
dfUser.columns

Index(['Periods', 'S+R', 'Material', 'CHA', 'CHANNEL', 'RETAIL/AFH', 'STORE',
       'Customer', 'Cust Name', 'NW (Ton)', 'Value (IDR)', 'Value (USD)',
       'Ship To', 'Ship To Name', 'Qty (Karton)', 'Sales Person', 'SO No',
       'GI Date', 'RDD', 'B_ArtCode', 'B_Carton Code', 'B_Material Text',
       'B_Brand', 'B_Brand-2', 'B_Category', 'B_Category-2', 'B_Category-3',
       'B_StatusSKU', 'B_Mill', 'B_Zone', 'B_Zone-2', 'B_Zone-3', 'B_Town',
       'D_NOND', 'B_Week', 'B_Week Rdd'],
      dtype='object')

In [152]:
#Take features that we need
dfUser2 = dfUser[['S+R','GI Date', 'Material', 'Ship To', 'Customer','NW (Ton)', 'Value (IDR)','Sales Person', 'SO No']]

In [153]:
dfUser2.head()

,S+R,GI Date,Material,Ship To,Customer,NW (Ton),Value (IDR),Sales Person,SO No
0,freegood,11.01.2018,69917620,1001551,1001551,0.035784,0.0,NaN,1029200331
1,freegood,18.01.2018,69916821,1001551,1001551,0.006400,0.0,NaN,1029200334
2,freegood,19.01.2018,69916820,1001551,1001551,0.012800,0.0,NaN,1029200335
3,freegood,23.01.2018,69916821,1001551,1001551,0.019200,0.0,NaN,1029200336
4,freegood,24.01.2018,69916582,1001551,1001551,0.012659,0.0,NaN,1029200332


In [154]:
#Change data type of GI Date to Date
dfUser2['GI Date'] = pd.to_datetime(dfUser2['GI Date'],dayfirst=True) 

C:\Users\01135710\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [155]:
#Take out FENNY RIANY LALENO value of Sales Person
dfUser3 = dfUser2[dfUser2['Sales Person'] != 'FENNY RIANY LALENO']

In [156]:
dfUser3.head()

,S+R,GI Date,Material,Ship To,Customer,NW (Ton),Value (IDR),Sales Person,SO No
0,freegood,2018-01-11,69917620,1001551,1001551,0.035784,0.0,NaN,1029200331
1,freegood,2018-01-18,69916821,1001551,1001551,0.006400,0.0,NaN,1029200334
2,freegood,2018-01-19,69916820,1001551,1001551,0.012800,0.0,NaN,1029200335
3,freegood,2018-01-23,69916821,1001551,1001551,0.019200,0.0,NaN,1029200336
4,freegood,2018-01-24,69916582,1001551,1001551,0.012659,0.0,NaN,1029200332


# Filter Data

In [157]:
#Function filter data per day
def excludeDataDay(data,c):
    data2 = data[(data['NW (Ton)'] > 0) & (data['Value (IDR)'] != 0) ]  #Exclude Free Goods
    data3 = data2[data2['Customer'] != 2064741]     #Exclude Paseo Lady
    data4 = data3[data3['GI Date'] == c]
    print("Data Dimension ", data4.shape)
    print("Sum Value ", sum(data4['Value (IDR)']))
    print("Sum Weight ", sum(data4['NW (Ton)']))
    return data4

In [158]:
#Function filter Interval data
def excludeIntervalData(data,c):
    data2 = data[(data['NW (Ton)'] > 0) & (data['Value (IDR)'] != 0) ]  #Exclude Free Goods
    data3 = data2[data2['Customer'] != 2064741]     #Exclude Paseo Lady
    data4 = data3[c]
    print("Data Dimension ", data4.shape)
    print("Sum Value ", sum(data4['Value (IDR)']))
    print("Sum Weight ", sum(data4['NW (Ton)']))
    return data4

# Find Diff 

In [180]:
#2018 Desember Data
c = (dfOur2['GI Date']>='2018-12-01') & (dfOur2['GI Date'] <= '2018-12-31')
d = (dfUser3['GI Date']>='2018-12-01') & (dfUser3['GI Date'] <= '2018-12-31')
print("dfOur Desc 2018 :")
dfOurDes18 = excludeIntervalData(dfOur2,c)
print()
print("dfUser Desc 2018 :")
dfUserDes18= excludeIntervalData(dfUser3,d)

dfOur Desc 2018 :


C:\Users\01135710\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Data Dimension  (20941, 8)
Sum Value  284811832517.0
Sum Weight  11274.550959999986

dfUser Desc 2018 :
Data Dimension  (24349, 9)
Sum Value  283296601243.0
Sum Weight  11220.03393600017


In [181]:
dfOurDes18.to_excel('ourDataDecember2018(Diff).xlsx')
dfUserDes18.to_excel('userDataDecember2018(Diff).xlsx')

In [182]:
#2018, 01 - 07 Desember Data
c = (dfOur2['GI Date']>='2018-12-01') & (dfOur2['GI Date'] <= '2018-12-07')
d = (dfUser3['GI Date']>='2018-12-01') & (dfUser3['GI Date'] <= '2018-12-07')
print("dfOur Desc 1-7 2018 :")
dfOurDes1817 = excludeIntervalData(dfOur2,c)
print()
print("dfUser Desc 1-7 2018 :")
dfUserDes1817= excludeIntervalData(dfUser3,d)

dfOur Desc 1-7 2018 :


C:\Users\01135710\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Data Dimension  (5605, 8)
Sum Value  62902131175.0
Sum Weight  2450.580776999993

dfUser Desc 1-7 2018 :
Data Dimension  (6101, 9)
Sum Value  62131996130.0
Sum Weight  2425.9697139999853


In [183]:
dfOurDes1817.to_excel('ourDataDecemberWeekOne2018(Diff).xlsx')
dfUserDes1817.to_excel('userDataDecemberWeekOne2018(Diff).xlsx')

In [184]:
#2018, 1 Desember Data
c = '2018-12-01'
d = '2018-12-01'
print("dfOur Desc 1 2018 :")
dfOurDes181 = excludeDataDay(dfOur2,c)
print()
print("dfUser Desc 1 2018 :")
dfUserDes181= excludeDataDay(dfUser3,d)

dfOur Desc 1 2018 :
Data Dimension  (297, 8)
Sum Value  3188018157.0
Sum Weight  124.903395

dfUser Desc 1 2018 :
Data Dimension  (334, 9)
Sum Value  3188018157.0
Sum Weight  124.90339500000002


In [185]:
dfOurDes181.to_excel('ourDataDecemberDayOne2018(Same).xlsx')
dfUserDes181.to_excel('userDataDecemberDayOne2018(Same).xlsx')

In [171]:
#2018, 2 Desember Data
c = '2018-12-02'
d = '2018-12-02'
print("dfOur Desc 2 2018 :")
dfOurDes18 = excludeDataDay(dfOur2,c)
print()
print("dfUser Desc 2 2018 :")
dfUserDes18= excludeDataDay(dfUser3,d)

dfOur Desc 2 2018 :
Data Dimension  (39, 8)
Sum Value  585204637.0
Sum Weight  24.60126500000001

dfUser Desc 2 2018 :
Data Dimension  (40, 9)
Sum Value  585204637.0
Sum Weight  24.601265000000005


In [186]:
#2018, 3 Desember Data
c = '2018-12-03'
d = '2018-12-03'
print("dfOur Desc 3 2018 :")
dfOurDes183 = excludeDataDay(dfOur2,c)
print()
print("dfUser Desc 3 2018 :")
dfUserDes183= excludeDataDay(dfUser3,d)

dfOur Desc 3 2018 :
Data Dimension  (587, 8)
Sum Value  6984620729.0
Sum Weight  276.33582300000006

dfUser Desc 3 2018 :
Data Dimension  (675, 9)
Sum Value  6984152549.0
Sum Weight  276.32655699999975


In [187]:
dfOurDes183.to_excel('ourDataDecemberDayThree2018(Same).xlsx')
dfUserDes183.to_excel('userDataDecemberDayThree2018(Same).xlsx')

In [174]:
#2018, 4 Desember Data
c = '2018-12-04'
d = '2018-12-04'
print("dfOur Desc 4 2018 :")
dfOurDes18 = excludeDataDay(dfOur2,c)
print()
print("dfUser Desc 4 2018 :")
dfUserDes18= excludeDataDay(dfUser3,d)

dfOur Desc 4 2018 :
Data Dimension  (1078, 8)
Sum Value  11015551392.0
Sum Weight  413.0799239999999

dfUser Desc 4 2018 :
Data Dimension  (1167, 9)
Sum Value  10890335748.0
Sum Weight  409.5943920000001


In [175]:
#2018, 5 Desember Data
c = '2018-12-05'
d = '2018-12-05'
print("dfOur Desc 5 2018 :")
dfOurDes18 = excludeDataDay(dfOur2,c)
print()
print("dfUser Desc 5 2018 :")
dfUserDes18= excludeDataDay(dfUser3,d)

dfOur Desc 5 2018 :
Data Dimension  (1003, 8)
Sum Value  14069250801.0
Sum Weight  557.5670699999998

dfUser Desc 5 2018 :
Data Dimension  (1135, 9)
Sum Value  13899128759.0
Sum Weight  551.787334
